# M2

#### Import Neccessary Packages
The following packages included in cb1020.yml are necessary for the model to run. 
These include classes of the mesa package, as well as the agent classes improted from the various agent class files.

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from Tumor_cells import Tumor_cells
import random

## Overview
Below is the implementation of the M2 agent. The agent is defined through its instance methods of def step, def migrate and def support_tumor_cells


### Key Attributes of the def __init__ instance:
- **Migration Probability (`prob_migrate`)**: Likelihood of moving to a new grid position during a step.
- **Death Probability (`prob_death`)**: Probability of dying during a step.
- **Support Growth Probability (`prob_support_growth`)**: Likelihood of supporting tumor cell growth.

### Key Attributes of the def __step__ instance: 
- **




In [ ]:

class M2(Agent):
    
    """
        Initializes an M2 macrophage agent.

        Args:
            unique_id (int): Unique identifier for the agent.
            model (Model): The model the agent belongs to.
            params (dict): Dictionary of model parameters.
    """
    def __init__(self, unique_id, position, model):
        super().__init__(unique_id, model)
        self.position = position
        self.prob_migrate = 0.4
        self.prob_death = 0.005
        self.prob_support_growth = 0.05
        self.alive = True
    
    def eat(self, val):
        self.model.eat_nutrition(val)
    """
        Executes one step of the macrophage's behavior:
        - Checks if the macrophage dies.
        - Attempts to migrate to a neighboring cell.
        - Supports the growth of nearby tumor cells with a given probability.
    """
    def step(self):
        # Check if macrophage should die
        if not self.alive:
            return
        self.eat(5)
        if self.random.random() < self.prob_death:
            self.alive = False
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
        if self.random.random() < self.prob_migrate:
            self.migrate()
        if self.random.random() < self.prob_support_growth:
            self.support_tumor_cells()

        # Attempt to migrate
    def migrate(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter only empty positions
        empty_positions = [pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]

        #Pick an empty position if there are any
        if len(empty_positions) > 0:
            new_position = self.random.choice(empty_positions)
            self.model.grid.move_agent(self, new_position)

        
# Check neighbors for tumor cells
    def support_tumor_cells(self):
        neighbors = self.model.grid.get_neighbors(self.position, moore=True, include_center=False)
        for neighbor in neighbors:
            if isinstance(neighbor, Tumor_cells):
            # Support tumor growth with probability
                if self.random.random() < self.prob_support_growth:
                # Create a new tumor cell in a random neighboring position
                    neighbors = self.model.grid.get_neighborhood(neighbor.position, moore=True, include_center=False)
                    tumor_cells = [cell for cell in neighbors if isinstance(cell, Tumor_cells)]
                    if tumor_cells:
                        tumor_cell = self.random.choice(tumor_cells) 
                        tumor_cell.set_proliferation_prob(2, "proportion")
                        tumor_cell.set_death_prob(0,"value")
                    
                    for agent in tumor_cells:
                        agent.set_angiogenesis_intensity(1)
                        #print("M2 Supperoted TUMOR PROLIFERATIOn")
    
    """
        Moves the macrophage to a random neighboring position.
    """
    def random_move(self):
        possible_steps = self.model.grid.get_neighborhood(self.position, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)